# FM를 이용한 추천시스템
- user, item 만 사용

| **구분**                  | **Explicit Feedback Data**                                         | **Implicit Feedback Data**                                             |
|---------------------------|--------------------------------------------------------------------|------------------------------------------------------------------------|
| **Data 종류**             | - Item에 대한 직접적인 평가, 취향, 선호도 등                       | - Platform에서 얻을 수 있는 데이터                                      |
|                           |  - 예: 평점, 구매 후기                                             | - User의 선호도/취향을 정확히 파악할 수 없으나, 그럴듯한 추론 가능         |
|                           |                                                                   |   - 예: 사용자 로그 분석, 검색 기록, 클릭 여부, 구매 여부, 장바구니 등      |
| **장점**                  | - 사용자가 직접 입력한 데이터이어서 품질이 좋음                     | - 다양한 방법으로 데이터를 얻을 수 있다                                  |
| **단점**                  | - 데이터를 구하기 어렵다                                           | - 데이터의 실제적 의미를 파악하기 어렵다                                 |
|                           | - 신뢰하기 어렵다                                                 | - 노이즈/결측치가 존재                                                  |
|                           | - 모든 사용자가 전부 참여하지 않는다                                | - 부정적인 feedback 얻기 어렵다                                          |
|                           |                                                                   | - 사용자의 행동에 대한 가중치/신뢰도 정도 파악 어려움                      |
|                           |                                                                   | - 데이터 품질에 대한 평가 기준이 불분명                                   |


### Factorization Machines
: `SVM`과 `Factorization Model` 들의 장점을 결합한 FM이라는 새로운 모델을 소개
- SVM과 마찬가지로 FM은 그 어떤 실수 값의 피처 벡터를 input으로 받아도 잘 작동하는 일반적인 예측기, 그러나 SVM과 다르게 Factorized Parameter를 이용하여 모든 Interaction(상호작용) 을 모델화하여 아주 희소한 상황에서도 Interaction들을 예측할 수 있다는 장점을 갖고 있다.
- 모델 수식 : $\hat{y}(x) = w_0 + \sum_{i=1}^n w_i x_i + \sum_{i=1}^n \sum_{j=i+1}^n \langle v_i, v_j \rangle x_i x_j$
    - $w_0$ : bias term
    - $w_i$ : i번째 feature의 weight (특성의 가중치)
    - $v_i$ : i번째 feature의 factorization vector (잠재 벡터)

### SVM (Support Vector Machine)
- 분류와 회귀 문제를 해결하는데 사용
- 목표 : 두 클래스 사이의 **최적의 결정 경계**를 찾아 데이터를 나누는 것
- 결정 경계란? : 데이터를 분리하는 선, 평면, 초평면 등을 의미
- Support vector : 결정 경계에 가장 가까운 데이터 포인트를 의미, 이 포인트들이 결정 경계를 정의하는 데 핵심적인 역할을 함

In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

## Data load

In [2]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/Users/jun/Library/Mobile Documents/com~apple~CloudDocs/Github/ai _recommendation _system/data/u.data', sep='\t', names=r_cols, encoding='latin-1')
ratings.head()

,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


## user encoding

In [3]:
user_dict = {}
for i in set(ratings['user_id']): # 유니크한 데이터
    user_dict[i] = len(user_dict)
n_user = len(user_dict)
n_user

943

In [4]:
user_dict

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 51: 50,
 52: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 91: 90,
 92: 91,
 93: 92,
 94: 93,
 95: 94,
 96: 95,
 97: 96,
 98: 97,
 99: 98,
 100: 99,
 101: 100,
 102: 101,
 103: 102,
 104: 103,
 105: 104,
 106: 105,
 107: 106,
 108: 107,
 109: 108,
 110: 109,
 111: 11

In [5]:
user_dict[196]

195

## Item encoding

In [7]:
item_dict = {}
start_point = n_user
for i in set(ratings['movie_id']):
    item_dict[i] = start_point + len(item_dict) # 유저의 개수에 더한 것
n_item = len(item_dict)

start_point += n_item
num_x = start_point               # Total number of x
ratings = shuffle(ratings, random_state=1)

In [8]:
n_item

1682

In [9]:
item_dict[196]

1138

In [10]:
item_dict

{1: 943,
 2: 944,
 3: 945,
 4: 946,
 5: 947,
 6: 948,
 7: 949,
 8: 950,
 9: 951,
 10: 952,
 11: 953,
 12: 954,
 13: 955,
 14: 956,
 15: 957,
 16: 958,
 17: 959,
 18: 960,
 19: 961,
 20: 962,
 21: 963,
 22: 964,
 23: 965,
 24: 966,
 25: 967,
 26: 968,
 27: 969,
 28: 970,
 29: 971,
 30: 972,
 31: 973,
 32: 974,
 33: 975,
 34: 976,
 35: 977,
 36: 978,
 37: 979,
 38: 980,
 39: 981,
 40: 982,
 41: 983,
 42: 984,
 43: 985,
 44: 986,
 45: 987,
 46: 988,
 47: 989,
 48: 990,
 49: 991,
 50: 992,
 51: 993,
 52: 994,
 53: 995,
 54: 996,
 55: 997,
 56: 998,
 57: 999,
 58: 1000,
 59: 1001,
 60: 1002,
 61: 1003,
 62: 1004,
 63: 1005,
 64: 1006,
 65: 1007,
 66: 1008,
 67: 1009,
 68: 1010,
 69: 1011,
 70: 1012,
 71: 1013,
 72: 1014,
 73: 1015,
 74: 1016,
 75: 1017,
 76: 1018,
 77: 1019,
 78: 1020,
 79: 1021,
 80: 1022,
 81: 1023,
 82: 1024,
 83: 1025,
 84: 1026,
 85: 1027,
 86: 1028,
 87: 1029,
 88: 1030,
 89: 1031,
 90: 1032,
 91: 1033,
 92: 1034,
 93: 1035,
 94: 1036,
 95: 1037,
 96: 1038,
 97: 1039,

- User Encoding과 Item Encoding은 데이터를 모델이 처리하기 쉽게 변환하는 전처리
    - 중복되지 않은 고유한 유저 ID와 아이템 ID를 한번만 카운트하여 라벨인코딩을 진행.

## Generate X Data

In [11]:
data = [] # 모델의 입력(feature)
y = [] # 각 데이터의 정답값(레이블), 평점(rating)에서 평균 평점(w0)을 뺀값
w0 = np.mean(ratings['rating'])

for i in range(len(ratings)):
    case = ratings.iloc[i]  # 행을 기준으로 추출
    x_index = []
    x_value = []
    x_index.append(user_dict[case['user_id']])     # User id encoding, user_id만 뽑아냄
    x_value.append(1)
    x_index.append(item_dict[case['movie_id']])    # Movie id encoding, movie_id만 뽑아냄
    x_value.append(1)
    
    data.append([x_index, x_value])
    y.append(case['rating'] - w0)
    
    # 진행도 확인
    if (i % 10000) == 0:
        print('Encoding ', i, ' cases...')

Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...
Encoding  40000  cases...
Encoding  50000  cases...
Encoding  60000  cases...
Encoding  70000  cases...
Encoding  80000  cases...
Encoding  90000  cases...


In [12]:
print(w0)

3.52986


In [13]:
data[:5] # data는 리스트, 처음 5개를 확인 / index가 507번인 유저(508번)가 1127번 영화를 평가

[[[29, 1722], [1, 1]],
 [[415, 1296], [1, 1]],
 [[35, 1261], [1, 1]],
 [[341, 1523], [1, 1]],
 [[30, 1261], [1, 1]]]

In [14]:
ratings.head()

,user_id,movie_id,rating,timestamp
85548,30,780,4,875060217
28997,416,354,4,893214333
4964,36,319,2,882157356
13453,342,581,3,875320037
32848,31,319,4,881547788


In [15]:
print(user_dict[508], item_dict[185]) # 인덱스인것을 확인가능

507 1127


## RMSE

In [16]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

## FM class
- 핵심 : SGD를 통해 파라미터 업데이트와 예측값 계산을 수행하는 `sgd()`메서드와 `predict()`메서드를 구현

In [17]:
class FM():
    def __init__(self, N, K, data, y, alpha, beta, train_ratio=0.75, iterations=100, tolerance=0.005, l2_reg=True, verbose=True):
        self.K = K                          # Number of latent factors
        self.N = N                          # Number of x (variables)
        self.n_cases = len(data)            # N of observations
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.l2_reg = l2_reg
        self.tolerance = tolerance
        self.verbose = verbose
        
        # w 초기화
        self.w = np.random.normal(scale=1./self.N, size=(self.N))
        # v 초기화
        self.v = np.random.normal(scale=1./self.K, size=(self.N, self.K))
        
        # Train/Test 분리
        cutoff = int(train_ratio * len(data))
        self.train_x = data[:cutoff]
        self.test_x = data[cutoff:]
        self.train_y = y[:cutoff]
        self.test_y = y[cutoff:]

    def test(self):                                     # Training 하면서 RMSE 계산
        # SGD를 iterations 숫자만큼 수행
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):
            rmse1 = self.sgd(self.train_x, self.train_y)        # SGD & Train RMSE 계산
            rmse2 = self.test_rmse(self.test_x, self.test_y)    # Test RMSE 계산
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                       # New best record
                best_RMSE = rmse2
                best_iteration = i
            elif (rmse2 - best_RMSE) > self.tolerance:  # RMSE is increasing over tolerance
                break
        print(best_iteration, best_RMSE)
        return training_process

    # Stochastic gradient descent를 통해 w, v 업데이트
    def sgd(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            x_idx = data[0]
            x_0 = np.array(data[1])     # xi axis=0 [1, 2, 3]
            x_1 = x_0.reshape(-1, 1)    # xi axis=1 [[1], [2], [3]]

            # biases (선형 항)
            bias_score = np.sum(self.w[x_idx] * x_0)

            # score 계산
            vx = self.v[x_idx] * (x_1)                      # v matrix * x
            sum_vx = np.sum(vx, axis=0)                     # sigma(vx)
            sum_vx_2 = np.sum(vx * vx, axis=0)              # ( v matrix * x )의 제곱
            
            # latent score : 상호작용 항, v를 활용해 상호작용 항을 계산
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

            # 예측값 계산 : 선형 항과 상호작용 항의 합 = 최종 예측값
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)
            error = y - y_hat
            
            # w, v 업데이트 , L2 regularization 수행하는 부분
            if self.l2_reg:     # regularization이 있는 경우
                self.w[x_idx] += error * self.alpha * (x_0 - self.beta * self.w[x_idx])
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1) - self.beta * self.v[x_idx])
            else:               # regularization이 없는 경우
                self.w[x_idx] += error * self.alpha * x_0
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1))
        return RMSE(y_data, y_pred)

    def test_rmse(self, x_data, y_data):
        y_pred = []
        for data , y in zip(x_data, y_data):
            y_hat = self.predict(data[0], data[1])
            y_pred.append(y_hat)
        return RMSE(y_data, y_pred)

    # 새로운 입력 데이터에 대해 예측값을 계산
    def predict(self, idx, x):
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1, 1)

        # biases
        bias_score = np.sum(self.w[idx] * x_0)

        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx, axis=0)
        sum_vx_2 = np.sum(vx * vx, axis=0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat


- `tolerance` 변수 의미
    - tolerance는 허용 오차 범위를 의미
    - 특정 반복(iteration)에서 모델의 Test RMSE가 이전 최적의 RMSE보다 증가하고, 그 증가폭이 tolerance 값 이상이면 학습을 **조기 종료(Early Stopping)** 하는 조건
    1. 조기 종료
    2. 과적합 방지
    3. 모델 안정화
- SGD, L2 정규화
    - L2 정규화 : 모델의 파라미터 값의 크기를 최소화하여 과적합을 방지하는 기법
    - 파라미터에 패널티를 추가, 손실 함수에 반영
    - 일반적으로 L2정규화는 가중치 w 또는 잠재 벡터 v의 제곱을 추가하여 과도한 파라미터 값이 학습되지 않도록 한다. 
- `verbose=True` : 코드의 출력 제어를 담당
    - 학습 과정에서의 진행 상황이나 중간 결과를 출력할지 여부를 결정하는 플래그 변수

In [18]:
K = 350
fm1 = FM(num_x, K, data, y, alpha=0.0014, beta=0.075, train_ratio=0.75, iterations=600, tolerance=0.0005, l2_reg=True, verbose=True)
result = fm1.test()

Iteration: 10 ; Train RMSE = 0.958508 ; Test RMSE = 0.965306
Iteration: 20 ; Train RMSE = 0.936596 ; Test RMSE = 0.949987
Iteration: 30 ; Train RMSE = 0.927653 ; Test RMSE = 0.944261
Iteration: 40 ; Train RMSE = 0.922810 ; Test RMSE = 0.941450
Iteration: 50 ; Train RMSE = 0.919670 ; Test RMSE = 0.939853
Iteration: 60 ; Train RMSE = 0.917142 ; Test RMSE = 0.938786
Iteration: 70 ; Train RMSE = 0.914398 ; Test RMSE = 0.937830
Iteration: 80 ; Train RMSE = 0.910358 ; Test RMSE = 0.936531
Iteration: 90 ; Train RMSE = 0.903293 ; Test RMSE = 0.934200
Iteration: 100 ; Train RMSE = 0.891087 ; Test RMSE = 0.930072
Iteration: 110 ; Train RMSE = 0.872987 ; Test RMSE = 0.924317
Iteration: 120 ; Train RMSE = 0.850235 ; Test RMSE = 0.918405
Iteration: 130 ; Train RMSE = 0.823505 ; Test RMSE = 0.913335
Iteration: 140 ; Train RMSE = 0.792361 ; Test RMSE = 0.909286
Iteration: 150 ; Train RMSE = 0.756715 ; Test RMSE = 0.906477
Iteration: 160 ; Train RMSE = 0.717313 ; Test RMSE = 0.905160
Iteration: 170 ; 

In [20]:
result # iteration, RMSE

[(0, np.float64(1.0949285950435885), np.float64(1.0630355265424536)),
 (1, np.float64(1.0487588858102246), np.float64(1.0312339531669965)),
 (2, np.float64(1.0214166481182019), np.float64(1.0116248583401133)),
 (3, np.float64(1.0034120464696952), np.float64(0.9984238600109714)),
 (4, np.float64(0.9906336747200578), np.float64(0.9889675449869344)),
 (5, np.float64(0.981066351486372), np.float64(0.9818739069556025)),
 (6, np.float64(0.9736096828837871), np.float64(0.976357997691296)),
 (7, np.float64(0.9676130765693209), np.float64(0.9719435994937725)),
 (8, np.float64(0.9626684787151902), np.float64(0.9683268515254007)),
 (9, np.float64(0.9585078614518088), np.float64(0.9653058191819203)),
 (10, np.float64(0.9549483822205113), np.float64(0.9627416387463259)),
 (11, np.float64(0.9518612355903924), np.float64(0.9605359527550442)),
 (12, np.float64(0.9491531062234743), np.float64(0.9586172527139096)),
 (13, np.float64(0.9467546927191419), np.float64(0.9569323241532972)),
 (14, np.float64(0